In [2]:
import os
os.environ['CUDA_DEVICE_ORDER'] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "7"

In [3]:
import numpy as np
import pickle as pic

from sklearn.model_selection import train_test_split

In [4]:
import deep_learning

In [6]:
if True:
    with open("../traces_and_splits/splitted_round_perm.pic", "rb") as r:
        X_training, X_val, X_extraction = pic.load(r)
    with open("../traces_and_splits/splitted_labels_1000000.pic", "rb") as r:
        y_training, y_val, y_extraction = pic.load(r)
else:
    with open("../traces_and_splits/traces_round_perm_only.pic", "rb") as r:
        traces_round_perm = pic.load(r)
    with open("../labels_pics/seeds_key_labels_1000000.pic", "rb") as r:
        _, _, rws_perms_labels, round_perms_labels, copy_perms_labels, rws_masks_labels, round_masks_labels = pic.load(r)

    X_total, y_total = deep_learning.prepare_data_dl(traces_round_perm, round_perms_labels, copy_perms_labels, round_masks_labels, rws_perms_labels, rws_masks_labels)

    profile, test = train_test_split(np.arange(X_total.shape[0]), train_size=750_000, random_state=0)

    X_profiling, X_extraction = X_total[profile], X_total[test]
    train, val = train_test_split(np.arange(X_profiling.shape[0]), test_size=0.1, random_state=0)
    X_training, X_val = X_profiling[train], X_profiling[val]

    with open("../traces_and_splits/splitted_round_perm.pic", "wb") as w:
        pic.dump((X_training, X_val, X_extraction), w)

    if False:
        y_profiling = {}
        y_training = {}
        y_val = {}
        y_extraction = {}
        for label in y_total.keys():
            print(label, end="\r")
            y_profiling[label], y_extraction[label] = y_total[label][profile], y_total[label][test]
            y_training[label], y_val[label] = y_profiling[label][train], y_profiling[label][val]

        with open("../traces_and_splits/splitted_labels_1000000.pic", "wb") as w:
            pic.dump((y_training, y_val, y_extraction), w)
    else:
        with open("../traces_and_splits/splitted_labels_1000000.pic", "rb") as r:
            y_training, y_val, y_extraction = pic.load(r)


In [10]:
resnet = deep_learning.ResNetSCA(network="orig_round_perm_only", epochs=1000)
try:
    deep_learning.check_file_exists("resnet_models/resnet_750000_round_perm.keras")
    from tensorflow.keras.models import load_model
    resnet.model = load_model("resnet_models/resnet_750000d_perm.keras")
except ValueError:
    pass

In [11]:
train_gen = deep_learning.DataGenerator(X_training, y_training)
val_gen = deep_learning.DataGenerator(X_val, y_val)

In [12]:
history = resnet.train_model(train_gen, val_gen, "resnet_models/resnet_750000_round_perm.keras", patience=10)

Epoch 1/1000
10547/10547 [==============================] - 481s 45ms/step - loss: 0.8163 - accuracy: 0.6730 - val_loss: 2.6509 - val_accuracy: 0.4042
Epoch 2/1000
10547/10547 [==============================] - 480s 46ms/step - loss: 0.6968 - accuracy: 0.7213 - val_loss: 2.9132 - val_accuracy: 0.4263
Epoch 3/1000
10547/10547 [==============================] - 480s 45ms/step - loss: 0.6681 - accuracy: 0.7332 - val_loss: 1.2960 - val_accuracy: 0.5595
Epoch 4/1000
10547/10547 [==============================] - 478s 45ms/step - loss: 0.6533 - accuracy: 0.7392 - val_loss: 0.7550 - val_accuracy: 0.7022
Epoch 5/1000
10547/10547 [==============================] - 472s 45ms/step - loss: 0.6434 - accuracy: 0.7438 - val_loss: 1.0355 - val_accuracy: 0.6178
Epoch 6/1000
10547/10547 [==============================] - 498s 47ms/step - loss: 0.6345 - accuracy: 0.7474 - val_loss: 0.9436 - val_accuracy: 0.6488
Epoch 7/1000
10547/10547 [==============================] - 672s 64ms/step - loss: 0.6283 - ac